## RU Metadata Extraction

In [1]:
import requests

ACCESS_TOKEN = '5lRvVDSnCTTXgdFWLuCN7HLAK2UWKjUbJwPCEiWJxirzVT3VfLsAeHnhflmt'
search_query = 'creators.affiliation:(+university +rockefeller)'

response = requests.get('https://zenodo.org/api/records/',
                        params={'q': search_query,
                                'access_token': ACCESS_TOKEN,
                                'size': 250, # should be 196, add headroom
                                'type' : 'dataset'
                                })

records = response.json()


Generate the necessary dataframe

In [2]:
import pandas as pd

### MAKE THE DATAFRAME
df = pd.json_normalize(records['hits']['hits'][0])
for i in range(1, len(records['hits']['hits'])):
    df_row = pd.json_normalize(records['hits']['hits'][i])
    # df_row = df_row.drop(columns = ['stats'])

    df = pd.concat([df, df_row])

### DROP UNWANTED COLUMNS

unwanted_cols = ['conceptrecid', 'recid', 'revision', 'files', 'owners', 'status', 'state', 'submitted', 'metadata.title', 'metadata.resource_type.title', 
                 'metadata.resource_type.type', 'metadata.communities', 'metadata.relations.version', 'links.self', 'links.doi', 'links.self_doi', 'links.self_doi_html', 
                 'links.parent', 'links.self_iiif_manifest', 'links.self_iiif_sequence', 'links.files', 'links.media_files', 'links.archive', 'links.archive_media', 
                 'links.latest', 'links.latest_html', 'links.versions', 'links.draft', 'links.reserve_doi', 'links.access_links', 'links.access_grants', 'links.access_users',
                 'links.access_request', 'links.access', 'links.communities', 'links.communities-suggestions', 'links.requests', 'stats.downloads', 'stats.unique_downloads',
                 'stats.views', 'stats.unique_views', 'stats.version_downloads', 'stats.version_unique_downloads', 'stats.version_unique_views', 'stats.version_views'
                ]

df = df.drop(columns=unwanted_cols)

### EXPAND THE AUTHOR COLUMNS

# def expand_creators(df, col_name = 'metadata.creators'):
#     max_dicts = df[col_name].apply(len).max()
#     for i in range(max_dicts):
#         name_col = f'name_{i+1}'
#         affiliation_col = f'affiliation_{i+1}'
#         orcid_col = f'orcid_{i+1}'

#         # Extract the i-th dictionary's data if it exists, otherwise set None
#         df[name_col] = df[col_name].apply(lambda x: x[i]['name'] if i < len(x) and 'name' in x[i] else None)
#         df[affiliation_col] = df[col_name].apply(lambda x: x[i]['affiliation'] if i < len(x) and 'affiliation' in x[i] else None)
#         df[orcid_col] = df[col_name].apply(lambda x: x[i]['orcid'] if i < len(x) and 'orcid' in x[i] else None)
    
#     return df.drop(columns=[col_name])


def expand_columns(df, col_name, fields):
    df[col_name] = df[col_name].apply(lambda x: x if isinstance(x, list) else [])

    # Find the maximum number of dictionaries in any row
    max_dicts = df[col_name].apply(len).max()

    # Iterate over the possible dictionary positions (1 to max_dicts)
    for i in range(max_dicts):
        for field in fields:
            # Create new column names dynamically
            new_col = f'{field}_{i+1}'
            
            # Extract the i-th dictionary's field data if it exists, otherwise set None
            df[new_col] = df[col_name].apply(lambda x: x[i].get(field) if i < len(x) else None)
    
    return df.drop(columns=[col_name])
    # return df





# df_expanded = expand_creators(df)
df_expanded = expand_columns(df, col_name = 'metadata.creators', fields = ['name', 'affiliation', 'orcid'])
df_expanded = expand_columns(df_expanded, col_name = 'metadata.related_identifiers', fields = ['identifier', 'relation'])
df_expanded = expand_columns(df_expanded, col_name = 'metadata.grants', fields = ['code', 'title', 'funder'])


### CONVERT HEADERS TO MATCH DIGITAL COMMONS



### SAVE TO .CSV

# df.to_csv('zenodo_raw.csv', index=False)

df_expanded.to_csv('zenodo_expanded_raw_RU.csv', index=False, encoding='utf-8-sig')

C:\Users\Claire\AppData\Local\Temp\ipykernel_36940\346447635.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col] = df[col_name].apply(lambda x: x[i].get(field) if i < len(x) else None)
C:\Users\Claire\AppData\Local\Temp\ipykernel_36940\346447635.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col] = df[col_name].apply(lambda x: x[i].get(field) if i < len(x) else None)
C:\Users\Claire\AppData\Local\Temp\ipykernel_36940\346447635.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually t

Explore the structure of the data.

In [3]:
type(records)
records.keys()
print(len(records['hits']['hits']))
records['hits']['hits'][0]['metadata'].keys()

print(records['hits']['hits'][0]['metadata'].keys())
print(records['hits']['hits'][1]['metadata'].keys())
print(records['hits']['hits'][2]['metadata'].keys())

109
dict_keys(['title', 'doi', 'publication_date', 'description', 'access_right', 'creators', 'keywords', 'related_identifiers', 'locations', 'resource_type', 'license', 'communities', 'relations', 'notes'])
dict_keys(['title', 'doi', 'publication_date', 'description', 'access_right', 'creators', 'resource_type', 'license', 'relations'])
dict_keys(['title', 'doi', 'publication_date', 'description', 'access_right', 'creators', 'keywords', 'related_identifiers', 'resource_type', 'license', 'communities', 'relations', 'notes'])


In [4]:
print(records['hits']['hits'][1]['metadata']['grants'][0]['title'])

KeyError: 'grants'

In [12]:
for i in range(len(records['hits']['hits'])):
    if 'grants' in records['hits']['hits'][i]['metadata']:
        print( i, records['hits']['hits'][i]['metadata'].keys())


1 dict_keys(['title', 'doi', 'publication_date', 'description', 'access_right', 'creators', 'contributors', 'related_identifiers', 'custom', 'resource_type', 'license', 'grants', 'relations'])
5 dict_keys(['title', 'doi', 'publication_date', 'description', 'access_right', 'creators', 'contributors', 'keywords', 'related_identifiers', 'resource_type', 'license', 'grants', 'relations'])
9 dict_keys(['title', 'doi', 'publication_date', 'description', 'access_right', 'creators', 'dates', 'language', 'custom', 'resource_type', 'license', 'grants', 'relations'])
13 dict_keys(['title', 'doi', 'publication_date', 'description', 'access_right', 'creators', 'keywords', 'language', 'resource_type', 'license', 'grants', 'relations', 'notes'])
45 dict_keys(['title', 'doi', 'publication_date', 'description', 'access_right', 'creators', 'contributors', 'dates', 'language', 'resource_type', 'journal', 'alternate_identifiers', 'license', 'grants', 'relations'])
55 dict_keys(['title', 'doi', 'publicatio

In [7]:
results = []
record_ids = []

for i in range(len(records['hits']['hits'])):
    c = []
    contributors = ''
    
    record_id = records['hits']['hits'][i]['id']
    record_ids.append(record_id)
    if 'resource_type' in records['hits']['hits'][i]['metadata']:
        resource_type = records['hits']['hits'][i]['metadata']['resource_type']['title']
    else:
        resource_type = "none"
    if 'title' in records['hits']['hits'][i]['metadata']:
        title = records['hits']['hits'][i]['metadata']['title']
    else:
        title = "none"
    if 'contributors' in records['hits']['hits'][i]['metadata']:
        for j in range(len(records['hits']['hits'][i]['metadata']['contributors'])):
            c.append(records['hits']['hits'][i]['metadata']['contributors'][j]['type']) 
        
        contributor_counts = Counter(c)
        
        count = 0
        for contrib in contributor_counts.keys():
            if (count > 0):
                contributors += ','
            contributors += contrib + "=" + str(contributor_counts[contrib])
            count += 1
    else:    
        contributors = 'none'
    
    results.append([record_id,title,resource_type,contributors])

print(record_ids)

[3998336, 12535061, 4996700, 4959790, 10214497, 10381745, 11043321, 10976403, 4283075, 11200960, 7233045, 7735119, 4768768, 7608044, 5498898, 5259624, 11198790, 6555001, 5483897, 3990020, 8192482, 4949298, 8222799, 8199844, 8199697, 4941488, 4941983, 7255538, 7246185, 11379028, 7026059, 7714308, 7706957, 7349256, 7352495, 5007081, 7930635, 3459753, 7443697, 7349228, 3240607, 7734867, 7352402, 10632541, 7709868, 13315567, 7352291, 7352955, 10815974, 10433763, 8105835, 8195997, 4961809, 5602160, 4312076, 11126860, 10256697, 10256829, 4994711, 13738007, 13137486, 4938523, 13138646, 13730345, 13138766, 4932970, 10530481, 3966446, 3860592, 4998079, 4391044, 4776758, 4937761, 4921694, 4546338, 5026405, 3999062, 4983276, 8303284, 13835101, 7878371, 4986426, 4948959, 4960638, 5009501, 11393437, 6426323, 5022345, 4998564, 5703708, 3750635, 3750641, 6426039, 6426141, 6425937, 5248631, 5523274, 3750643, 3750637, 3750633, 3750639, 3750631, 3750629, 11280861, 5712332, 7883685, 4938137, 11167377, 11